In [1]:
import requests
import telebot
import moment
from datetime import datetime
from dotenv import dotenv_values
config = dotenv_values("../.env")



In [10]:

mydb = mysql.connector.connect(
    host=config["DBHOST"],
    user=config["DBUSER"],
    password=config["DBPASS"],
    database=config["DBNAME"]
)


NameError: name 'mysql' is not defined

In [ ]:


mycursor = mydb.cursor(dictionary=True)

mycursor.execute("SELECT * from countries WHERE name = 'Andorra'")

myresult = mycursor.fetchall()

print(myresult)

[{'id': 2931, 'name': 'Andorra', 'image_path': 'https://cdn.sportmonks.com/images/countries/png/short/ad.png', 'continent': 'Europe', 'sub_region': 'Southern Europe', 'world_region': 'EMEA', 'fifa': 'AND', 'iso': 'AND', 'iso2': 'AD', 'longitude': '1.5762332677841187', 'latitude': '42.5506591796875', 'flag_emoji': '🇦🇨'}]


In [79]:
for iso in flags:
  sql = f"UPDATE countries SET flag_emoji = '{flags[iso]['flag']}' WHERE iso2 = '{iso.upper()}'"
  mycursor.execute(sql)

mydb.commit()


In [16]:

def __fetch_fixture_by_id(fixture_id):
    # CODE SHOULD BE CHANGED TO FINDING THE FIXTURE FROM DYNAMODB
    res = requests.get(
        "https://gist.githubusercontent.com/VishalBty/c3bd3bae0d4fe8e1a6ccaae1e5f7dc0c/raw/2584294579bd4c42e725fb04b0f2d72af2c22782/fixture.json")
    return res.json()


def __fetch_strategy_by_id(strategy_id):
    # CODE SHOULD BE CHANGED TO FINDING THE STRATEGY FROM MYSQL
    res = requests.get(
        "https://gist.githubusercontent.com/VishalBty/7d82150efee726414a0b0b5e1d31bc61/raw/6406a50ee3b35fcfc9589e4e6d0f61c970d58752/strategy.json")
    return res.json()


def __fetch_user_by_id(user_id):
    # CODE SHOULD BE CHANGED TO FINDING THE USER FROM MYSQL
    res = requests.get(
        "https://gist.githubusercontent.com/VishalBty/7d82150efee726414a0b0b5e1d31bc61/raw/6406a50ee3b35fcfc9589e4e6d0f61c970d58752/strategy.json")
    return res.json()


def __field_for_probability(rule):
    return f'probability.{rule["code"]}'


def __field_for_odds(rule):
    return f'pre_odds.{rule["code"]}'


def __field_for_team(rule, team):
    return f'{team}.{rule[rule["location"]]}'


def __field_for_aggregate_stats(rule):
    if rule["team"] == "home":
        field = __field_for_team(rule, "home")
    elif rule["team"] == "away":
        field = __field_for_team(rule, "away")
    else:
        field = [__field_for_team(rule, "home"),
                 __field_for_team(rule, "away")]
    return field


def __format_stats_as_telegram_message(pre_match_data):
    lines = []
    for item in pre_match_data:
        value = item["value"]
        label = item["label"]
        line = f"<b>{label}</b>: {value}"
        lines.append(line)
    return "\n".join(lines)


def __get_prematch_data(fixture):
    pre_match_data = []
    locations = {
        "home": "Home",
        "away": "Away",
        "overall": "Overall",
    }
    teams = {
        "home": "Home",
        "away": "Away",
        "both": "Both Teams",
    }
    for rule in strategy["strategy_prematch_rules"]:
        label = rule["label"]
        if rule["category"] == "probability":
            fields = __field_for_probability(rule)
        elif rule["category"] == "odds":
            fields = __field_for_odds(rule)
        else:
            location = locations[rule["location"]]
            team = teams[rule["team"]]
            label += f" - {team} - {location} "
            fields = __field_for_aggregate_stats(rule)

        if not isinstance(fields, list):
            field = fields
        else:
            field = fields[0]
        value = fixture[field]
        pre_match_data.append({"value": value, "label": label})
    return pre_match_data


def __create_pre_match_message(fixture, strategy, stats_message, utcoffset=0, locale="en-gb"):
    outcomes_text = strategy["outcomes"]
    #   .map(outcome => capitalize(outcome.split("_").join(" ")))
    #   .join(",");
    league_name = fixture["league_name"]
    flag_emoji = fixture["flag_emoji"]
    country_name = fixture["country_name"]
    league_text = f"{flag_emoji or '🏆'} <b>{country_name} {league_name}</b>"

    title = strategy["title"]
    home_name = fixture["home_name"]
    away_name = fixture["away_name"]
    note = strategy["note"]
    unix_timestamp = fixture["timestamp"] * 1000

    match_time = moment.unix(
        unix_timestamp, utc=True).strftime("%d %b %Y %I:%M")
    lines = [
        f"🔔 <b>{title}</b>",
        "",
        f"🆚 <b>{home_name}</b> v <b>{away_name}</b>",
        "",
        league_text,
        "",
        f"<b>🥅 Desired Outcome:</b> {outcomes_text}",
        "",
        stats_message,
        "",
        f"""<b>Match Time:</b> {match_time}"""
    ]
    if note:
        note_text = f"\n<b>💭 My Note</b>\n{note}"
        lines.append(note_text)

    message = "\n".join(lines)
    return message


def __send_telegram_alert(chat_id, message):
    TOKEN = config["TELEGRAMBOTTOKEN"]
    bot = telebot.TeleBot(TOKEN)
    return bot.send_message(chat_id, message,  parse_mode="HTML")


In [17]:

fixture_id = 18213330
strategy_id = 35
user_id = 12

user = {"chat_id": 342833991} #__fetch_user_by_id(user_id)
strategy = __fetch_strategy_by_id(strategy_id)
fixture = __fetch_fixture_by_id(fixture_id)
chat_id = user["chat_id"]

In [18]:
pre_match_data = __get_prematch_data(fixture)
stats_message = __format_stats_as_telegram_message(pre_match_data)

In [19]:
message = __create_pre_match_message(fixture, strategy, stats_message)

In [20]:
__send_telegram_alert(chat_id=chat_id, message=message)
